In [8]:
import numpy as np
from IPython.display import clear_output

def display_board(board):
    """
    Displays the Connect 4 board from a 6x7x2 representation.
    Uses 'X' for player +1 and 'O' for player -1.
    """
    clear_output()
    
    horizontal_line = '-' * (7 * 5 + 8)
    blank_line = '|' + ' ' * 5
    blank_line *= 7
    blank_line += '|'
    
    print('   0     1     2     3     4     5     6')
    print(horizontal_line)
    
    for row in range(6):
        print(blank_line)
        this_line = '|'
        for col in range(7):
            if board[row, col, 0] == 1:  # Player +1
                this_line += '  X  |'
            elif board[row, col, 1] == 1:  # Player -1
                this_line += '  O  |'
            else:  # Empty space
                this_line += '     |'
        print(this_line)
        print(blank_line)
        print(horizontal_line)
    
    print('   0     1     2     3     4     5     6')

In [9]:
import numpy as np

def update_board(board_temp, color, column):
    """
    Updates the board (6x7x2) by placing a checker in the specified column.
    
    Parameters:
    - board_temp: 6x7x2 NumPy array representing the board
    - color: 'plus' for +1, 'minus' for -1
    - column: Integer (0-6) representing the column where the piece is dropped
    
    Returns:
    - Updated 6x7x2 board with the new piece added
    """
    board = board_temp.copy()
    
    # Find the lowest available row in the given column
    for row in range(5, -1, -1):  # Start from bottom row
        if board[row, column, 0] == 0 and board[row, column, 1] == 0:  # Check if empty
            if color == 'plus':
                board[row, column, 0] = 1  # Set +1 in first channel
                board[row, column, 1] = 0
            else:
                board[row, column, 0] = 0
                board[row, column, 1] = 1  # Set -1 in second channel
            return board  # Return updated board
    
    # If column is full, return the board unchanged
    return board


In [10]:
def check_for_win(board, col):
    """
    Checks for a win in the 6x7x2 board.

    Parameters:
    - board: 6x7x2 NumPy array
    - col: Integer (0-6) indicating the last column where a checker was dropped

    Returns:
    - 'v-plus', 'v-minus' for vertical win
    - 'h-plus', 'h-minus' for horizontal win
    - 'd-plus', 'd-minus' for diagonal win
    - 'nobody' if no win
    """
    nrow, ncol = 6, 7
    
    # Find the row of the last played move
    for row in range(6):
        if board[row, col, 0] == 1 or board[row, col, 1] == 1:
            break  # Found the last placed checker
    
    # Identify which player made the move
    player = "plus" if board[row, col, 0] == 1 else "minus"
    check_channel = 0 if player == "plus" else 1
    
    def check_direction(delta_row, delta_col):
        """Counts consecutive checkers in a specific direction"""
        count = 0
        r, c = row, col
        while 0 <= r < nrow and 0 <= c < ncol and board[r, c, check_channel] == 1:
            count += 1
            r += delta_row
            c += delta_col
        return count

    # **Check vertical (↓)**
    if check_direction(1, 0) >= 4:
        return f'v-{player}'
    
    # **Check horizontal (← →)**
    left_count = check_direction(0, -1)  # Count leftwards
    right_count = check_direction(0, 1)  # Count rightwards
    if left_count + right_count - 1 >= 4:
        return f'h-{player}'

    # **Check diagonal (↘ ↖)**
    down_right = check_direction(1, 1)
    up_left = check_direction(-1, -1)
    if down_right + up_left - 1 >= 4:
        return f'd-{player}'

    # **Check diagonal (↙ ↗)**
    down_left = check_direction(1, -1)
    up_right = check_direction(-1, 1)
    if down_left + up_right - 1 >= 4:
        return f'd-{player}'

    return 'nobody'  # No win detected


In [11]:
def swap_board(board):
    """
    Swap [0,1] to [1,0] and [1,0] to [0,1] in a given board only if player == -1.
    """

    board = np.array(board)  # Ensure it's a NumPy array

    # Create masks
    mask_01 = (board[:,:,0] == 0) & (board[:,:,1] == 1)  # Find [0,1]
    mask_10 = (board[:,:,0] == 1) & (board[:,:,1] == 0)  # Find [1,0]

    # Swap values
    board[mask_01] = [1, 0]
    board[mask_10] = [0, 1]

    return board  # Return the modified or original board

In [12]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LeakyReLU

# Custom object dictionary
custom_objects = {'LeakyReLU': LeakyReLU}

# Load the model with custom objects
cnn_model = load_model('cnn_model3.h5', custom_objects=custom_objects)

# Display the model summary
cnn_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_13 (Conv2D)              │ (None, 6, 7, 64)       │         3,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 6, 7, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 6, 7, 64)       │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 6, 7, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 3, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 3, 3, 128)      │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 3, 3, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 3, 3, 128)      │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 3, 3, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 1, 1, 128)      │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 1, 1, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 1, 1, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 1, 1, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 1, 1, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 1, 1, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,192,328 (4.55 MB)

 Trainable params: 1,190,791 (4.54 MB)

 Non-trainable params: 1,536 (6.00 KB)

 Optimizer params: 1 (8.00 B)

In [13]:
def play_game():
    # Initialize an empty board
    board = np.zeros((6, 7, 2), dtype=int)

    # Ask the user if they want to go first
    user_first = input("Do you want to go first? (y/n): ").strip().lower()
    user_player = 'minus' if user_first == 'y' else 'plus'

    # Initialize the winner
    winner = 'nobody'

    # Display the initial empty board
    display_board(board)

    while winner == 'nobody':
        if user_player == 'minus':
            # User's turn
            move = int(input('Pick a move (0-6) for player minus: '))
            board = update_board(board, 'minus', move)
            display_board(board)
            winner = check_for_win(board, move)
            user_player = 'plus'  # Switch to CNN's turn
        else:
            # CNN's turn
            swapped_board = swap_board(board)
            cnn_prediction = cnn_model.predict(swapped_board[np.newaxis, ...])
            cnn_move = np.argmax(cnn_prediction)
            board = update_board(board, 'plus', cnn_move)
            display_board(board)
            winner = check_for_win(board, cnn_move)
            user_player = 'minus'  # Switch to user's turn

    if winner == 'v-plus' or winner == 'h-plus' or winner == 'd-plus':
        print('The winner is CNN')
    else:
        print('The winner is YOU!!!!')

# PLAY GAME!!!

In [ ]:
play_game()